In [61]:
import numpy as np
from sklearn import neural_network

In [8]:
cc = np.load('cluster_centers.npy')
labels = np.load('labels.npy')
tac = np.load('train_art_clusters.npy')

In [4]:
print cc.shape

(200, 50)


In [9]:
print tac.shape

(1048575, 3)


In [6]:
print labels.shape

(233286,)


In [62]:
# relies on chopping off column headers ahead of time. 
artists = np.loadtxt('data/artists.csv', dtype='string', delimiter=',')
print artists
profiles = np.loadtxt('data/profiles_fixed.csv', dtype='string', delimiter=',')
print profiles.shape

# fixing erroneous, irrelevant ages like 1002
max_age = 120
# total = 0
# count = 0
for p in profiles:
    if p[2] != '' and (int(p[2]) >= max_age or int(p[2]) < 0):
        # 22 is the average age of those who put down ages within a reasonable range
        p[2] = '22'

art_dict = {}
count = 0
for a in artists:
    art_dict[a[0]] = count
    count += 1
    
prof_dict = {}
count = 0
for a in profiles:
    prof_dict[a[0]] = count
    count += 1
    
nation_dict = {}
count = 0
for p in profiles:
    if p[3] not in nation_dict:
        nation_dict[p[3]] = None
num_nations = len(nation_dict)
count = 0
for key in nation_dict.keys():
    nation_dict[key] = [0] * num_nations
    nation_dict[key][count] = 1
    count += 1

[['03098741-08b3-4dd7-b3f6-1b0bfa2c879c' 'Liars']
 ['69c4cc43-8163-41c5-ac81-30946d27bb69' 'CunninLynguists']
 ['7a2e6b55-f149-4e74-be6a-30a1b1a387bb' 'The Desert Sessions']
 ..., 
 ['8974da95-e631-45aa-8fd7-aa0c2795f997' 'Harry Gregson-Williams']
 ['8067c102-4996-42bc-9980-06ce2e644eae' 'Soul Coughing']
 ['39c2a93d-9afa-4a22-9bba-c087ab056e1c' 'Jefferson Airplane']]
(233286, 4)


In [63]:
train = np.loadtxt('data/train_fixed.csv', dtype='string', delimiter=',')

In [64]:
def make_one_hot(length, n):
    lst = [0] * length
    lst[n] = 1
    return lst

In [26]:
pre_X = []
pre_Y = []
count = 0
for t in train[:1048575]:
# for t in train[:10]:
    p = prof_dict[t[0]]
    
    temp_x = []
    temp_x += make_one_hot(200, labels[p])
    temp_x += make_one_hot(50, int(tac[count][1]))
    
    if profiles[p][1] == 'f':
        temp_x += make_one_hot(3, 0)
    elif profiles[p][1] == 'm':
        temp_x += make_one_hot(3, 1)
    else:
        temp_x += make_one_hot(3, 2)
        
    if profiles[p][2] == '':
        temp_x += make_one_hot(121, 120)
    else:
        temp_x += make_one_hot(121, int(profiles[p][2]))
        
    temp_x += nation_dict[profiles[p][3]]
        
    pre_X.append(temp_x)
    
    pre_Y.append(int(t[2]))
    
    count += 1

In [27]:
X = np.array(pre_X)
Y = np.array(pre_Y)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[554  81 708 ..., 230  66 284]


In [65]:
def mse(Y, Y_cv):
    diff = Y_cv - Y
    d_sq = np.square(diff)
    mse_res = np.mean(d_sq)
    return mse_res

In [ ]:
MLP = neural_network.MLPRegressor(max_iter=200, verbose=True)
MLP.fit(X, Y)

Iteration 1, loss = 319097.67500054
Iteration 2, loss = 299663.07319329
Iteration 3, loss = 294541.12457682
Iteration 4, loss = 291878.43829243
Iteration 5, loss = 290117.92648822
Iteration 6, loss = 288792.43514116
Iteration 7, loss = 287775.12603029
Iteration 8, loss = 286936.67536880
Iteration 9, loss = 286264.29826525
Iteration 10, loss = 285683.73858215
Iteration 11, loss = 285219.90430750
Iteration 12, loss = 284799.68837347
Iteration 13, loss = 284444.97253445
Iteration 14, loss = 284112.93953250
Iteration 15, loss = 283786.50531307
Iteration 16, loss = 283518.14733265
Iteration 17, loss = 283246.69709349
Iteration 18, loss = 282998.42572028
Iteration 19, loss = 282761.22661641
Iteration 20, loss = 282537.95932575
Iteration 21, loss = 282322.33041414
Iteration 22, loss = 282131.33000825
Iteration 23, loss = 281897.45422551
Iteration 24, loss = 281711.91342968
Iteration 25, loss = 281548.05709230
Iteration 26, loss = 281345.99843318
Iteration 27, loss = 281175.87307260
Iteration 

In [36]:
Y_cv = MLP.predict(X[-1000:])
print mse(Y[-1000:], Y_cv)

144942.48414


In [37]:
import pickle
pickle.dump(MLP, open("mlp_model.p", "wb"))

In [66]:
MLP = pickle.load('mlp_model.p')

In [67]:
test = np.loadtxt('data/test.csv', dtype='string', delimiter=',')

In [38]:
elt_list = []
for t in train:
    elt_list.append(float(t[2]))
elist = sorted(elt_list)
global_median = elist[train.shape[0] / 2]

In [ ]:
print tac[0]
def get_artist_labels():
    d = {}
    for i in range(len(tac)):
        if tac[i][1] not in d:
            d[train[i][1]] = int(tac[i][1])
    return d
art_labels = get_artist_labels()

In [71]:
import csv

soln_file = 'data/mlp_real.csv'

with open('data/test.csv', 'r') as test_fh:
# with open('data/train.csv', 'r') as test_fh:
    test_reader = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_reader, None)
    
#     print test.shape

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])
        
        count = 0
        for row in test_reader:
            id     = row[0]
            user   = row[1]
            artist = row[2]
            
            if artist in art_labels and user in prof_dict:
                    p = prof_dict[user]

                    temp_x = []
                    temp_x += make_one_hot(200, labels[p])
                    temp_x += make_one_hot(50, art_labels[artist])

                    if profiles[p][1] == 'f':
                        temp_x += make_one_hot(3, 0)
                    elif profiles[p][1] == 'm':
                        temp_x += make_one_hot(3, 1)
                    else:
                        temp_x += make_one_hot(3, 2)

                    if profiles[p][2] == '':
                        temp_x += make_one_hot(121, 120)
                    else:
                        temp_x += make_one_hot(121, int(profiles[p][2]))

                    temp_x += nation_dict[profiles[p][3]]

                    x = np.array([temp_x])

                    soln_csv.writerow([id, MLP.predict(x)])
            else:
                soln_csv.writerow([id, global_median + 120.0])
                
            if count % 10000 == 0:
                print count

AttributeError: 'numpy.ndarray' object has no attribute 'predict'